### Лаба 2. Content-based рекомендательная система образовательных курсов – Spark Dataframes

Задача

По имеющимся данным портала eclass.cc построить content-based рекомендации по образовательным курсам. Запрещено использовать библиотеки pandas, sklearn и аналогичные.

[[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
[21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
[16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
[11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
[16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], 
[13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
import pyspark.sql.functions as f
from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab2")
         .getOrCreate())

### load data

In [ ]:
data = spark.read.json('/labs/slaba02/DO_record_per_line.json')

In [ ]:
data.show(5)

### tf-idf
TF — это term frequency: по сути, сколько раз слово встречается в этом документе. Если мы сделаем такой word count по каждому документу, то получим вектор, который как-то характеризует этот документ.
DF – document frequency: по сути, число документов, в которых есть вхождение этого слова. Мы хотим "штрафовать" слово за частое появление в документах, поэтому делаем инверсию этой величины – буква I в TFIDF.

In [ ]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")
ht = HashingTF(numFeatures=10000, inputCol="words", outputCol='features')

#### IDF transformation

In [ ]:
idf = IDF(inputCol='features', outputCol="idf_features", minDocFreq=5)

### pipeline

In [ ]:
pipeline = Pipeline(stages=[tokenizer, ht, idf])
pipe = pipeline.fit(data)
pipe_data = pipe.transform(data).cache()

### filter by courses

In [ ]:
target_films = [23126, 21617, 16627, 11556, 16704, 13702]
targets = pipe_data.filter(f.col('id').isin(target_films)).select('id', 'idf_features').collect()
df = pipe_data.select('id', 'idf_features').collect()

### metric

In [ ]:
all_ = []
metric = {}
for i in range(len(targets)):
    u = targets[i][1]
    si = {}
    for k in range(len(df)):
        v = df[k][1]
        # в качестве метрики для ранжирования — косинус угла между TFIDF-векторами для разных курсов.
        si[df[k][0]] =  v.dot(u) / (v.norm(2) * u.norm(2))
    metric[targets[i][0]] = si
    all_.append(metric)

In [ ]:
# Для каждого такого ключа в качестве значения задается массив рекомендованных курсов, состоящий из их id, 
# отсортированных по убыванию метрики. При равенстве значений метрики курсов сортируются лексикографически по названию.
rec=[]
result = {}
for i, t in zip(range(len(metric)), target_films) :
    sorted_ = sorted(metric[t].items(), key=lambda item: item[1], reverse=True)[:11]
    rec = []
    for k in range(len(sorted_)):
        rec.append(sorted_[k][0])
    result[t] = rec

for i in target_films:
    result[i] = list(set(result[i]) - set([i]))

In [ ]:
result

### make json

In [ ]:
with open("lab02.json", "w", encoding="utf-8") as file:
    json.dump(result, file)

### put into server

In [ ]:
!hdfs dfs -put lab02.json /user/olga.pogodina

In [ ]:
spark.stop()